###  Comparison between best Deep Learning Models for Tabular data  using *mushroom_cleaned* dataset

I'm going to use these models:
- #### TabNet
- #### TabTransforner
- #### SAINT
- #### TF-Transformer

For each model I will get the execution time, loss, Test  Accuracy and their parameters archiving a minimum of .85 in val accuracy.

Every model have a training-testing size --> 80-20 with a random_seed=42, an *ADAM optimizer* and a *BCEWwithLogistLoss loss function*

#### Imports

In [13]:
import numpy as np
import pandas as pd
import time
# Preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#
import torch
import torch.nn as nn
# Models
from tab_transformer_pytorch import TabTransformer
from saint_pytorch import SAINT
# Metrics
from sklearn.metrics import accuracy_score

In [12]:
time_Model = {}

#### Load Data

In [3]:
dataset = pd.read_csv('mushroom_cleaned.csv', sep=',')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54035 entries, 0 to 54034
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cap-diameter     54035 non-null  int64  
 1   cap-shape        54035 non-null  int64  
 2   gill-attachment  54035 non-null  int64  
 3   gill-color       54035 non-null  int64  
 4   stem-height      54035 non-null  float64
 5   stem-width       54035 non-null  int64  
 6   stem-color       54035 non-null  int64  
 7   season           54035 non-null  float64
 8   class            54035 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 3.7 MB


##### Data treatment common for each model

In [14]:
categorical_cols = ['cap-diameter','cap-shape', 'gill-attachment', 'gill-color', 'stem-width', 'stem-color']
continuous_cols = ['stem-height', 'season']
label_col = 'class'
#
labelEncodersTabTransformer = {}
#
for col in categorical_cols: 
    le = LabelEncoder();
    dataset[col] = le.fit_transform(dataset[col])
    labelEncodersTabTransformer[col] = le
# Split data into 'data' and 'labels'
X = dataset.drop(label_col, axis=1)
y = dataset[label_col]
# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Separate categorical and continous features
X_train_cat =   X_train[categorical_cols].values
X_train_cont =  X_train[continuous_cols].values
#
X_test_cat =    X_test[categorical_cols].values
X_test_cont =   X_test[continuous_cols].values
#
# Convert to PyTorch Tensors
X_train_cat_tensor =    torch.tensor(X_train_cat,       dtype=torch.long)
X_train_cont_tensor =   torch.tensor(X_train_cont,      dtype=torch.float)
y_train_tensor =        torch.tensor(y_train.values,    dtype=torch.float)
# 
X_test_cat_tensor =     torch.tensor(X_test_cat, dtype=torch.long)
X_test_cont_tensor =    torch.tensor(X_test_cont, dtype=torch.float)
y_test_tensor =         torch.tensor(y_test.values, dtype=torch.float)

### TabTransformer

#### Define the model + Optimizer + Loss function

In [7]:
# Parameters
NUM_UNIQUE_CATEGORIES = [dataset[col].nunique() for col in categorical_cols]
NUM_CONTINOUS  = len(continuous_cols)
DIM = 32
DIM_OUT = 1
DEPTH = 6
HEADS = 8
ATTN_DROPOUT = .1
FF_DROPOUT = .1

In [8]:
modelTabTrans = TabTransformer(
    categories=NUM_UNIQUE_CATEGORIES,
    num_continuous=NUM_CONTINOUS,
    dim=DIM,
    dim_out=DIM_OUT,
    depth=DEPTH,
    heads=HEADS,
    attn_dropout=ATTN_DROPOUT,
    ff_dropout=FF_DROPOUT
)

In [9]:
# Optimizer + Loss function
optimizer = torch.optim.Adam(modelTabTrans.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

#### TabTransformer Train

In [ ]:
loss_TabTrans = [] # loss per epoch
ini_time = time.time()
modelTabTrans.train()
for epoch in range(15):
    optimizer.zero_grad()
    output = modelTabTrans(X_train_cat_tensor, X_train_cont_tensor)
    loss = criterion(output.squeeze(), y_train_tensor)
    loss.backward()
    optimizer.step()
    loss_TabTrans.append(loss.item())
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
time_Model['TabTransformer'] = time.time() - ini_time

#### TabTransformer Evaluation

In [ ]:
modelTabTrans.eval()
with torch.no_grad():
    # Test Loss
    test_output = modelTabTrans(X_test_cat_tensor, X_test_cont_tensor)
    test_loss = criterion(test_output.squeeze(), y_test_tensor)
    print(f'Test Loss: {test_loss.item()}')
    # Accuracy
    test_preds = torch.sigmoid(test_output).squeeze().round()
    accuracy = accuracy_score(y_test_tensor.numpy(), y_test_tensor)
    print(f'Test Accuracy: {accuracy * 100:.4f}%')

### TF-Transformer

No tenemos una implementación directa de TF-Transformer en PyTorch o Skleaern.

### SAINt